In [1]:
import sys
import numpy as np
import os
import SimpleITK as sitk
sys.path.remove('/home/sentic/.local/lib/python3.6/site-packages')

import torch
torch.backends.cudnn.benchmark=True

device_id = 1
torch.cuda.set_device(device_id)

root = "/home/sentic/MICCAI/data/train/"
use_gpu = True
n_epochs = 300
batch_size = 1
use_amp = False
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import LambdaLR

from torch.utils.data import DataLoader

from model import LargeCascadedModel
from dataset import BraTS
from losses import DiceLoss, DiceLossLoss
from tqdm import tqdm_notebook, tqdm
path = "./checkpoints/checkpoint_80.pt"


model = LargeCascadedModel(inplanes_encoder_1=4, channels_encoder_1=16, num_classes_1=3,
                           inplanes_encoder_2=7, channels_encoder_2=32, num_classes_2=3)
model.load_state_dict(torch.load(path, map_location='cuda:1')['state_dict'])
BraTSDataset = BraTS(root=root, phase="train", desired_depth=160, desired_height=240, desired_width=240)
dataloader = DataLoader(BraTSDataset, batch_size=batch_size, shuffle=False)

if use_gpu:
    model = model.to("cuda")

diceLoss = DiceLossLoss()
model.eval()
threshold = 0.5

all_losses = 0

In [3]:
dir_patients = "/home/sentic/MICCAI/data/train/"
out_dir = "./final_masks_train/"

threshold_wt = 0.7
threshold_tc = 0.6 
threshold_et = 0.7
threshold_num_pixels_et = 0

for ix, loader in tqdm(enumerate(dataloader)):
    volume, mask, contour, patient = loader
    if use_gpu:
        volume = volume.to("cuda")
        mask = mask.to("cuda")
        contour = contour.to("cuda")

    with torch.no_grad():
        _, _, decoded_region3, _ = model(volume)
        decoded_region3 = decoded_region3.detach().cpu().numpy()
        decoded_region3 = decoded_region3.squeeze()

    np_array = decoded_region3[:, :155, : , :]
    path_patient = os.path.join(dir_patients, patient[0], patient[0] + "_flair.nii.gz")
    image_patient = sitk.ReadImage(path_patient)
    direction = image_patient.GetDirection()
    origin = image_patient.GetOrigin()
    spacing = image_patient.GetSpacing()

    seg_image = np.zeros((155, 240, 240))

    label_1 = np_array[2, :, :, :] # where the enhanced tumor is
    location_pixels_et = np.where(label_1 > threshold_et)
    num_pixels_et = location_pixels_et[0].shape[0]
    label_2 = np_array[1, :, :, :] # locatia 1-urilor si 4-urilor
    label_3 = np_array[0, :, :, :] # locatia 1 + 2 + 4
    
    if num_pixels_et > threshold_num_pixels_et: # if there are at least num of pixels
        label_1[label_1 > threshold_et] = 1 # put them in et category
    else:
        label_1[label_1 > threshold_et] = 0 # don't put them
        label_2[location_pixels_et] = 1 # but put them on tumor core
        
    label_1[label_1 < threshold_et] = 0
    location_1 = np.where(label_1 != 0)
    seg_image[location_1] = 4
    
    label_2[label_2 > threshold_tc] = 1
    label_2[label_2 < threshold_tc] = 0
    location_2 = np.where((label_2 != 0) & (label_1 == 0))
    seg_image[location_2] = 1

    
    label_3[label_3 > threshold_wt] = 1
    label_3[label_3 < threshold_wt] = 0
    location_3 = np.where((label_3 != 0) & (label_2 == 0))
    seg_image[location_3] = 2

    seg_Image = sitk.GetImageFromArray(seg_image)
    seg_Image.SetDirection(direction)
    seg_Image.SetOrigin(origin)
    seg_Image.SetSpacing(spacing)


    sitk.WriteImage(seg_Image, os.path.join(out_dir, patient[0] + ".nii.gz"))


369it [19:01,  3.09s/it]
